# Análisis de GW170817
Un ejemplo para hallar la SNR con una plantilla Post-Newtoniana utilizando la libreria pycbc

In [5]:
import sys
!{sys.executable} -m pip install pycbc ligo-common --no-cache-dir

In [6]:
import numpy as np
import matplotlib.mlab as mlab
from pycbc.waveform import get_fd_waveform, get_td_waveform
from pycbc.waveform import td_approximants, fd_approximants
from pycbc.detector import Detector
from pycbc import waveform
from pycbc.filter import matched_filter
from scipy import signal
from scipy.signal import butter, filtfilt
from scipy.interpolate import interp1d
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.projections as mprj
import h5py

In [7]:
# Formato para las graficas
LNWDT=1.3; FNT=18
plt.rcParams['lines.linewidth'] = LNWDT; plt.rcParams['font.size'] = FNT
#plt.rcParams['xtick.direction'] = 'out'; plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['xtick.top'] = 'on'; plt.rcParams['ytick.right'] = 'on'
#plt.rcParams['xtick.minor.visible'] = True; plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['font.family'] = 'Times New Roman'
#plt.rcParams['font.serif'] = 'Times'
rc('mathtext', fontset='stix')

In [8]:
#### Constants cgs #####
G = 6.67408*10**-8
c = 2.99792458*10**10
Msun = 1.98855*10**33
tSol = G*Msun/c**3

parsec = 3.08568025e18
d_source = 1.e6*parsec

In [9]:
# funcion para whitening
def whiten(strain, interp_psd, dt):
    Nt = len(strain)
    freqs = np.fft.rfftfreq(Nt, dt)

    hf = np.fft.rfft(strain)
    norm = 1./np.sqrt(1./(dt*2))
    white_hf = hf / np.sqrt(interp_psd(freqs)) * norm
    white_ht = np.fft.irfft(white_hf, n=Nt)
    return white_ht

In [35]:
from google.colab import drive
drive.mount('/content/drive')

detectors = ['H1','L1']

#strain, time, channel_dict = rl.loaddata(fileName)
data_H1 = h5py.File('/content/drive/MyDrive/ColabNotebooks/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', "r")
data_L1 = h5py.File('/content/drive/MyDrive/ColabNotebooks/H-H1_LOSC_CLN_4_V1-1187007040-2048.hdf5', "r")

data_in = [data_H1, data_L1]

T = 32*2
tevent = 1842

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
strain_H1 = data_H1["strain"]["Strain"][()]
dt_H1 = data_H1["strain"]["Strain"].attrs["Xspacing"]

strain_L1 = data_L1["strain"]["Strain"][()]
dt_L1 = data_L1["strain"]["Strain"].attrs["Xspacing"]

In [37]:
if dt_H1 == dt_L1:
    delta_t = dt_H1
else:
    delta_t = dt_H1
    print ('time arrays sampling is not equal')
fs = int(1/delta_t)
fmin = 30.
fmax = 1600.

In [13]:
mchirp = 1.197
q = 1
nu = q / (1.+q)**2
mtot = mchirp / nu**(3/5.)
m1 = 1.3758
m2 = 1.3758
lambda1 = 300
lambda2 = 300*q
print('q  = {0:.2f}'.format(q))
print('m1 = {0:.4f}, m2 = {1:.4f}'.format(m1, m2))
print('m total = {0:.2f}'.format(mtot))
print('mchirp det = {0:.3f}'.format(mchirp))
print((m1*m2/(m1+m2)**2)**(3/5.)*(m1+m2))

q  = 1.00
m1 = 1.3758, m2 = 1.3758
m total = 2.75
mchirp det = 1.197
1.1977034649828076


In [38]:
hp, hc = get_td_waveform(approximant='SpinTaylorT4', mass1=m1, mass2=m2,
                         f_lower=80, delta_t=1./fs,
                         lambda1=300, lambda2=10,
                         phase_order=-1,
                         amplitude_order=-1)